In [249]:
import dare
import random
import time
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from SISA_lib import SISA

In [250]:
data = pd.read_csv("cleaned_loan.csv")
data

,Pekerjaan,Jenis Pinjaman,Plafon (Rp.),Kondisi Joint Income,Kondisi Pisah Harta,Usia,Hasil IDEB SLIK OJK,Tujuan Pemilikan Properti,Lama bekerja,Penghasilan,...,Down Payment,Asset lainnya yang dimiliki,Bidang Usaha,Ukuran Perusahaan,Kondisi Sertifikat Agunan Utama,Jenis Bangunan Utama,Kondisi Properti Utama,Keputusan,Kelayakan Dokumen Pendukung,Keputusan Akhir
0,KARYAWAN,"PEMBELIAN BEKAS / SECONDARY, RUMAH TINGGAL","Plafon (< 514,750,000)",TIDAK JOINT INCOME,TIDAK PISAH HARTA,21 - 35,Data Tidak Tersedia,Rumah Tinggal (Rumah ke 1),"≥ 2, < 5",SINGLE : ≥RP 10 JUTA,...,>= 15%,DEPOSITO/INVES/EMAS ≥ LIMIT KREDIT,"Keuangan, asuransi, perbankan",Cakupan area : nasional,"SHM, SERTIPIKAT PECAH",Rumah Tinggal,READY STOK,GREEN,LAYAK,Diterima
1,PENGUSAHA,"PEMBELIAN BEKAS / SECONDARY, RUMAH TINGGAL","Plafon (> 1,340,960,436)",TIDAK JOINT INCOME,PISAH HARTA,36 - 45,"Kol 1, Tidak pernah menunggak",Rumah Tinggal (Rumah ke 1),≥ 5,MENIKAH : ≥RP 20 JUTA,...,>= 15%,DEPOSITO/INVES/EMAS ≥ LIMIT KREDIT,Lainnya,Cakupan area : nasional,"SHM, SERTIPIKAT PECAH",Rumah Tinggal,READY STOK,GREEN,LAYAK,Diterima
2,KARYAWAN,"TAKE OVER + TOP UP KPR, RUMAH TINGGAL","Plafon (> 1,340,960,436)",JOINT INCOME,TIDAK PISAH HARTA,46 - 55,"Kol 1, Tidak pernah menunggak",Rumah Tinggal (Rumah ke 1),≥ 5,MENIKAH : ≥RP 20 JUTA,...,>= 15%,PROPERTI/KENDARAAN ≥ LIMIT KREDIT,KLINIK AKUPUNTIR,CAKUPAN AREA : LOKAL,"SHM, SERTIPIKAT PECAH",Rumah Tinggal,SEDANG DIHUNI,GREEN,LAYAK,Diterima
3,PENGUSAHA,"PEMBELIAN BEKAS / SECONDARY, RUMAH TINGGAL","Plafon (894,632,536 - 1,340,960,436)",JOINT INCOME,TIDAK PISAH HARTA,36 - 45,"Kol 1, Tidak pernah menunggak",RUMAH TINGGAL (RUMAH KE 2/3),≥ 5,MENIKAH : ≥RP 20 JUTA,...,>= 15%,PROPERTI/KENDARAAN < 100% LIMIT KREDIT,PERDAGANGAN,CAKUPAN AREA : LOKAL,"SHM, SERTIPIKAT PECAH",RUKO/RUKAN,READY STOK,GREEN,LAYAK,Diterima
4,KARYAWAN,"PEMBELIAN BEKAS / SECONDARY, RUMAH TINGGAL","Plafon (< 514,750,000)",JOINT INCOME,TIDAK PISAH HARTA,21 - 35,Data Tidak Tersedia,Rumah Tinggal (Rumah ke 1),"≥ 2, < 5","SINGLE : ≥RP 6 JUTA, < RP. 10 JUTA",...,>= 15%,KAS DAN NON KAS ≥ LIMIT KREDIT,COLOUR SALON,CAKUPAN AREA : LOKAL,"SHGB, SERTIPIKAT PECAH",Rumah Tinggal,SEDANG DIHUNI,YELLOW,LAYAK,Diterima
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,KARYAWAN,"Pembelian Baru / Primary, Rumah Tinggal > Tipe 70","Plafon (> 1,340,960,436)",TIDAK JOINT INCOME,TIDAK PISAH HARTA,21 - 35,"Kol 1, Tidak pernah menunggak",Rumah Tinggal (Rumah ke 1),"≥ 2, < 5",SINGLE : ≥RP 20 JUTA,...,>= 15%,Properti/kendaraan < 50% plafon,"INFORMASI, TELEKOMUNIKASI DAN KOMPUTER",Public listed,MOU - Sertipikat Induk,Rumah Tinggal,Indent,YELLOW,TIDAK,Ditolak
196,KARYAWAN,"Pembelian Baru / Primary, Rumah Tinggal Tipe 2...","Plafon (< 514,750,000)",JOINT INCOME,TIDAK PISAH HARTA,21 - 35,Data Tidak Tersedia,Rumah Tinggal (Rumah ke 1),≥ 5,"MENIKAH : ≥RP 10 JUTA, <RP 20 JUTA",...,>= 15%,DEPOSITO/INVES/EMAS < 50% plafon,Lainnya,Multinasional,"SHGB, SERTIPIKAT PECAH",Rumah Tinggal,Indent,GREEN,TIDAK,Ditolak
197,KARYAWAN,"Pembelian Baru / Primary, Rumah Tinggal Tipe 2...","Plafon (< 514,750,000)",TIDAK JOINT INCOME,TIDAK PISAH HARTA,21 - 35,"Kol 1, Tidak pernah menunggak",Rumah Tinggal (Rumah ke 1),"≥ 2, < 5","SINGLE : ≥RP 6 JUTA, < RP. 10 JUTA",...,>= 15%,DEPOSITO/INVES/EMAS < 50% plafon,Lainnya,CAKUPAN AREA : LOKAL,MOU - Sertipikat Induk,Rumah Tinggal,Indent,YELLOW,TIDAK,Ditolak
198,PENGUSAHA,"Pembelian Baru / Primary, Ruko / Rukan","Plafon (> 1,340,960,436)",TIDAK JOINT INCOME,PISAH HARTA,56 - 65,"Kol 1, Tidak pernah menunggak",Rumah Tinggal (Rumah ke 1),≥ 5,MENIKAH : ≥RP 20 JUTA,...,>= 15%,null < 50% plafon,Lainnya,Multinasional,MOU - Sertipikat Induk,RUKO/RUKAN,Indent,GREEN,TIDAK,Ditolak


In [251]:
labelEncoder = {}
for col in data.columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    labelEncoder[col] = le
data

,Pekerjaan,Jenis Pinjaman,Plafon (Rp.),Kondisi Joint Income,Kondisi Pisah Harta,Usia,Hasil IDEB SLIK OJK,Tujuan Pemilikan Properti,Lama bekerja,Penghasilan,...,Down Payment,Asset lainnya yang dimiliki,Bidang Usaha,Ukuran Perusahaan,Kondisi Sertifikat Agunan Utama,Jenis Bangunan Utama,Kondisi Properti Utama,Keputusan,Kelayakan Dokumen Pendukung,Keputusan Akhir
0,0,1,2,1,1,0,0,2,1,5,...,1,2,14,1,6,2,1,0,0,0
1,1,1,3,1,0,1,8,2,2,3,...,1,2,15,1,6,2,1,0,0,0
2,0,9,3,0,1,2,8,2,2,3,...,1,9,12,0,6,2,2,0,0,0
3,1,1,1,0,1,1,8,1,2,3,...,1,7,19,0,6,1,1,0,0,0
4,0,1,2,0,1,0,0,2,1,8,...,1,5,2,0,3,2,2,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,6,3,1,1,0,8,2,1,7,...,1,10,6,4,0,2,0,2,1,1
196,0,7,2,0,1,0,0,2,2,2,...,1,1,15,3,3,2,0,0,1,1
197,0,7,2,1,1,0,8,2,1,8,...,1,1,15,0,0,2,0,2,1,1
198,1,5,3,1,0,3,8,2,2,3,...,1,12,15,3,0,1,0,0,1,1


In [252]:
data = data.drop_duplicates()
data

,Pekerjaan,Jenis Pinjaman,Plafon (Rp.),Kondisi Joint Income,Kondisi Pisah Harta,Usia,Hasil IDEB SLIK OJK,Tujuan Pemilikan Properti,Lama bekerja,Penghasilan,...,Down Payment,Asset lainnya yang dimiliki,Bidang Usaha,Ukuran Perusahaan,Kondisi Sertifikat Agunan Utama,Jenis Bangunan Utama,Kondisi Properti Utama,Keputusan,Kelayakan Dokumen Pendukung,Keputusan Akhir
0,0,1,2,1,1,0,0,2,1,5,...,1,2,14,1,6,2,1,0,0,0
1,1,1,3,1,0,1,8,2,2,3,...,1,2,15,1,6,2,1,0,0,0
2,0,9,3,0,1,2,8,2,2,3,...,1,9,12,0,6,2,2,0,0,0
3,1,1,1,0,1,1,8,1,2,3,...,1,7,19,0,6,1,1,0,0,0
4,0,1,2,0,1,0,0,2,1,8,...,1,5,2,0,3,2,2,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,6,3,1,1,0,8,2,1,7,...,1,10,6,4,0,2,0,2,1,1
196,0,7,2,0,1,0,0,2,2,2,...,1,1,15,3,3,2,0,0,1,1
197,0,7,2,1,1,0,8,2,1,8,...,1,1,15,0,0,2,0,2,1,1
198,1,5,3,1,0,3,8,2,2,3,...,1,12,15,3,0,1,0,0,1,1


In [253]:
X = data.drop(["Keputusan Akhir"], axis=1)
y = data["Keputusan Akhir"]

In [254]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [281]:
X_train

,Pekerjaan,Jenis Pinjaman,Plafon (Rp.),Kondisi Joint Income,Kondisi Pisah Harta,Usia,Hasil IDEB SLIK OJK,Tujuan Pemilikan Properti,Lama bekerja,Penghasilan,...,Jangka Waktu Kredit,Down Payment,Asset lainnya yang dimiliki,Bidang Usaha,Ukuran Perusahaan,Kondisi Sertifikat Agunan Utama,Jenis Bangunan Utama,Kondisi Properti Utama,Keputusan,Kelayakan Dokumen Pendukung
46,1,10,1,1,1,0,8,2,1,3,...,2,1,8,19,1,6,1,2,0,0
87,1,1,2,1,0,0,0,2,2,7,...,2,1,12,23,0,3,2,1,0,0
169,0,7,1,0,1,1,0,1,2,3,...,0,1,12,13,0,0,2,0,0,1
84,1,1,0,1,0,3,8,1,2,3,...,1,1,9,10,1,6,2,2,0,0
164,1,7,0,0,1,2,8,2,1,3,...,1,0,12,23,3,6,2,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,1,1,0,0,1,0,8,2,1,3,...,0,1,5,6,0,1,2,0,0,0
114,0,4,0,1,1,2,8,2,2,3,...,1,1,12,14,1,0,0,4,2,0
14,0,4,0,1,1,0,8,2,1,5,...,0,0,10,15,0,0,0,4,2,0
100,1,5,1,0,1,1,0,1,2,3,...,0,1,11,13,1,0,1,0,0,0


In [255]:
X_train_base = X_train.copy()
y_train_base = y_train.copy()
X_test_base = X_test.copy()

In [256]:
#SISA train and test dataset formats
X_train_SISA = X_train.values
y_train_SISA = y_train.values
X_test_SISA = X_test.values

In [257]:
#DaRE Forest train dataset
X_train_dare = np.array(X_train.values.tolist())
y_train_dare = np.array(y_train.values.tolist())
X_test_dare = np.array(X_test.values.tolist())

In [258]:
#make model for each machine learning model

In [259]:
#Base Model using RandomForest
tree_model = RandomForestClassifier(n_estimators=50,max_depth=5, random_state=42)
tree_model.fit(X_train_base, y_train_base)

RandomForestClassifier(max_depth=5, n_estimators=50, random_state=42)

In [260]:
#predict using XGBoost model
y_pred_tree = tree_model.predict(X_test_base)
accuracy_tree = accuracy_score(y_pred_tree, y_test)
print(f"Akurasi Model Decision Tree : {accuracy_tree}")

Akurasi Model Decision Tree : 0.9629629629629629


In [261]:
#SISA Model akurasi by shard
shard = 2
accuracy = 1
tertinngi = 0
accuracy_tertinngi = 0
exp_results = pd.DataFrame(columns=['Shard', 'Accuracy'])

In [262]:
while shard <= 6:
    sisa_by_shard = SISA(shard)
    sisa_by_shard .train(X_train_SISA, y_train_SISA)
    y_pred = sisa_by_shard .predict(X_test_SISA)
    accuracy = accuracy_score(y_pred, y_test)
    exp_results = pd.concat([exp_results, pd.DataFrame([{'Shard': shard, 'Accuracy': accuracy}])], ignore_index=True)
    if (accuracy > accuracy_tertinngi):
        accuracy_tertinngi = accuracy
        tertinngi = shard
    shard += 1

In [263]:
#show sisa accuracy by shard
data_result = pd.DataFrame(exp_results)
data_result

,Shard,Accuracy
0,2,0.981481
1,3,0.962963
2,4,0.870370
3,5,0.888889
4,6,0.944444


In [264]:
print(tertinngi)

2


In [265]:
#Base on result below, SISA model will use 4 shard for expriment
sisa = SISA(2)
sisa.train(X_train_SISA, y_train_SISA)

In [266]:
y_pred_SISA = sisa.predict(X_test_SISA)
accuracy_SISA = accuracy_score(y_pred_SISA, y_test)
print(f"Akurasi Model SISA : {accuracy_SISA}")

Akurasi Model SISA : 0.9814814814814815


In [267]:
#DaRE Forest Model
dafo = dare.Forest(n_estimators=50, max_depth=5, k = 5, topd = 0, random_state=42)
dafo.fit(X_train_dare, y_train_dare)

In [268]:
#Predict using DaRE Forest Model
y_pred_dare = dafo.predict(X_test_dare)
accuracy_dare = accuracy_score(y_pred_dare, y_test)
print(f"Akurasi Model DaRE Forest : {accuracy_dare}")

Akurasi Model DaRE Forest : 0.9629629629629629


In [269]:
test_proba_dare = np.array(X_test_dare)
dafo.predict_proba(test_proba_dare)

array([[0.21391518, 0.78608482],
       [1.        , 0.        ],
       [0.06982927, 0.93017073],
       [0.97979894, 0.02020106],
       [0.11926155, 0.88073845],
       [0.69121029, 0.30878971],
       [0.20745934, 0.79254066],
       [0.92181563, 0.07818437],
       [0.65514814, 0.34485186],
       [0.73305023, 0.26694977],
       [0.70685293, 0.29314707],
       [0.01027451, 0.98972549],
       [1.        , 0.        ],
       [0.16372222, 0.83627778],
       [0.96223637, 0.03776363],
       [0.25498645, 0.74501355],
       [0.05771612, 0.94228388],
       [0.76554092, 0.23445908],
       [0.27226434, 0.72773566],
       [0.17831595, 0.82168405],
       [0.70071029, 0.29928971],
       [0.70650432, 0.29349568],
       [0.11662505, 0.88337495],
       [0.2509188 , 0.7490812 ],
       [0.95651045, 0.04348955],
       [0.23833004, 0.76166996],
       [0.1300683 , 0.8699317 ],
       [0.10712348, 0.89287652],
       [0.13262505, 0.86737495],
       [0.1594618 , 0.8405382 ],
       [0.

In [270]:
test_proba_base = np.array(X_test_base)
tree_model.predict_proba(test_proba_base)

c:\Users\pilha\anaconda3\envs\TA\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array([[0.2001539 , 0.7998461 ],
       [0.99641026, 0.00358974],
       [0.10137374, 0.89862626],
       [0.97260295, 0.02739705],
       [0.10163573, 0.89836427],
       [0.72458017, 0.27541983],
       [0.26067222, 0.73932778],
       [0.90821479, 0.09178521],
       [0.68607648, 0.31392352],
       [0.75007362, 0.24992638],
       [0.74411971, 0.25588029],
       [0.01624553, 0.98375447],
       [0.99641026, 0.00358974],
       [0.20005683, 0.79994317],
       [0.92088645, 0.07911355],
       [0.2877093 , 0.7122907 ],
       [0.07860539, 0.92139461],
       [0.77914109, 0.22085891],
       [0.42075949, 0.57924051],
       [0.15168769, 0.84831231],
       [0.72759977, 0.27240023],
       [0.75745269, 0.24254731],
       [0.1383553 , 0.8616447 ],
       [0.3071146 , 0.6928854 ],
       [0.90557247, 0.09442753],
       [0.27064737, 0.72935263],
       [0.13737382, 0.86262618],
       [0.22219913, 0.77780087],
       [0.13347991, 0.86652009],
       [0.12745819, 0.87254181],
       [0.

In [271]:
#Perbandingan label untuk masing-masing model

In [272]:
comparation_pred_model = pd.DataFrame({
    'Decision Tree': y_pred_tree,
    'SISA': y_pred_SISA,
    'DaRE Forest': y_pred_dare
})

In [273]:
comparation_pred_model

,Decision Tree,SISA,DaRE Forest
0,1,1,1
1,0,0,0
2,1,1,1
3,0,0,0
4,1,1,1
5,0,0,0
6,1,1,1
7,0,0,0
8,0,0,0
9,0,0,0


#### Melakukan proses penghapusan secara random untuk masing-masing model

In [274]:
#Create dataFrame for each model
base_model = pd.DataFrame(columns=['Jumlah data', 'Accuracy', 'Time'])
SISA_model = pd.DataFrame(columns=['Jumlah data', 'Accuracy', 'Time'])
dare_model = pd.DataFrame(columns=['Jumlah data', 'Accuracy', 'Time'])
comparition_model = pd.DataFrame(columns=['Jumlah data', 'Accuracy Base', 'Accuracy SISA', 'Accuracy DaRE','Time Base', 'Time SISA', 'Time DaRE'])

In [275]:
#Create function to delete base model data
def delete_base_data(X_train_base, y_train_base, index): 
    X_data = X_train.iloc[index].tolist()
    data_to_delete = X_train_base.apply(lambda row: all(elem in row.values for elem in X_data), axis=1)
    X_train_base = X_train_base[~data_to_delete]
    y_train_base = y_train_base[~data_to_delete]
    return X_train_base, y_train_base

In [276]:
#create variabel for process
deleted_index = []
jumlah_data = len(X_train)
max_random_number = len(X_train) - 1
y_pred_comparison_flag = 0

In [277]:
while len(deleted_index) < max_random_number:
    #get new index
    new_index = False
    while new_index == False:
        index = random.randint(0, max_random_number - 1)
        if index not in deleted_index:
            deleted_index.append(index)
            break
    jumlah_data -= 1
    #Base Model with decreasing dataset
    try:
        start_time_base = time.time()
        X_train_base, y_train_base = delete_base_data(X_train_base, y_train_base,index)
        tree_model.fit(X_train_base, y_train_base)
        training_time_base = time.time() - start_time_base

        #prediction using base model with new data
        y_pred_tree = tree_model.predict(X_test_base)
        accuracy_tree = accuracy_score(y_pred_tree, y_test)

        #add to dataframe
        base_model = pd.concat([base_model, pd.DataFrame([{'Jumlah data': jumlah_data,'Accuracy': accuracy_tree, "Time" : training_time_base}])], ignore_index=True)
    except:
        print("Penghapusan data mencapai maksimal")
        break

    #delete elemen by sisa
    try:
        start_time_sisa = time.time()
        sisa.delete([X_train_SISA[index]])
        training_time_sisa = time.time() - start_time_sisa

        #prediction using model with deleted data
        y_pred_SISA = sisa.predict(X_test_SISA)
        accuracy_SISA = accuracy_score(y_pred_SISA, y_test)

        #add to dataframe
        SISA_model = pd.concat([SISA_model, pd.DataFrame([{'Jumlah data': jumlah_data,'Accuracy': accuracy_SISA, "Time" : training_time_sisa}])], ignore_index=True)
    except:
        print("salah satu data pada shard, sudah habis, silahkan train ulang model dari awal")
        # SISA_model = pd.concat([SISA_model, pd.DataFrame([{'Jumlah data': jumlah_data,'Accuracy': "-", "Time" : "-"}])], ignore_index=True)
        accuracy_SISA = "-"
        training_time_sisa = "-"
    
    #delete elemen by dareForest
    try:
        start_time_dare = time.time()
        dafo.delete(index)
        training_time_dare = time.time() - start_time_dare

        #prediction using model with deleted data
        y_pred_dare = dafo.predict(X_test_dare)
        accuracy_dare = accuracy_score(y_pred_dare, y_test)

        #add to dataframe
        dare_model = pd.concat([dare_model, pd.DataFrame([{'Jumlah data': jumlah_data,'Accuracy': accuracy_dare, "Time" : training_time_dare}])], ignore_index=True)
    except Exception as e:
        print(e)

    #comparisson model
    comparition_model = pd.concat([comparition_model, pd.DataFrame([{
        'Jumlah data' : jumlah_data,
        'Accuracy Base' : accuracy_tree,
        'Accuracy SISA' : accuracy_SISA,
        'Accuracy DaRE' : accuracy_dare,
        'Time Base' : training_time_base,
        'Time SISA' : training_time_sisa,
        'Time DaRE' : training_time_dare,
    }])], ignore_index=True)

    #comparisoon y_pred for each model
    if accuracy_tree < 0.7 and y_pred_comparison_flag == 0:
        y_pred_comparison_flag = 1
        y_pred_comparation = pd.DataFrame({
            'Data ke -': jumlah_data,
            'Decision Tree': y_pred_tree,
            'SISA': y_pred_SISA,
            'DaRE Forest': y_pred_dare,
            'y_test' : y_test
        })


Penghapusan data mencapai maksimal


In [278]:
comparition_model.to_csv("hasil_eksperiment/exp_2/hasil_eksperiment.csv", index=False)

In [279]:
y_pred_comparation.to_csv("hasil_eksperiment/exp_2/y_pred_compare.csv", index=False)